In [1]:
import subprocess
from astropy.table import Table
import os
import csv
#in the furture make packages and import common functions
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from pyraf import gwm
from astropy.io import ascii
import numpy as np
from astropy.stats import sigma_clip
from astropy.io import fits
from astropy.io import ascii
import subprocess

In [2]:
from pyraf import iraf
from iraf import obsutil
from iraf import psfmeasure
from iraf import phot
from iraf import daofind
from iraf import daophot
from iraf import psf
from iraf import artdata
from iraf import mkobjects
from iraf import starlist

In [5]:
imagename = '/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits'
ext =  79
diffImage = '/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits[{}]'.format(ext)
#diffImage = '/home/mj1e16/Simages/diff_79_2.fits'
SImage= '/home/mj1e16/Simages/testsimage.fits' #?


In [6]:
outDir = '/home/mj1e16/iraf/outputs/' # for iraf output files ONLY
imageDir = '/home/mj1e16/Simages/irafImages/' # for images generated during process ONLY
simageDir = '/home/mj1e16/Simages/' # directory for simulated images
irafDir = '/home/mj1e16/iraf/' #Directory containing iraf installation
starlistDir = '/home/mj1e16/starlists/'

In [16]:
def extractFitsExtension(imagename,extension,outname):
    
    hdu_list = fits.open(imagename)
    im = hdu_list[extension].data
    hdu_list.writeto(outname)

In [17]:
imname = '/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits'
ext = 79
outname = '/home/mj1e16/Simages/regIm_79.fits'
extractFitsExtension(imname,ext,outname)

In [7]:
def mkobjectspluspsf(imagename,diffimage,diffPlusSim,ext,section,minMag,maxMag,Xmax=1000,Xmin=0,Ymax=1000,Ymin=1000,thresh=50000,nsegs=5,goodpsf='',spatialDist='uniform',zeropoint=25.):
    
    daoout = outDir+'{}_{}daofindone'.format(ext,section)
    photout = outDir+'{}_{}photone'.format(ext,section)
    psfout = outDir+'{}_{}psfone'.format(ext,section)
    groupout = outDir+'{}_{}groupfileone'.format(ext,section)
    opstout = outDir+'{}_{}optsone'.format(ext,section)
    starfieldsect = outDir+'starfield_{}.dat'.format(section)
    imseg = imageDir+'imageseg{}.fits'.format(section)
    
    hdu = fits.open(imagename)
    print(imagename)
    image = hdu[ext].section[Xmin:Xmax,Ymin:Ymax]
    hdu = fits.PrimaryHDU(image)
    hdu.writeto(imseg,clobber=True)
    
    with open(starfieldsect,'w') as f:
        f.write("")
    
    daofind(imseg,output=daoout,sigma=1.0,threshold=thresh)
    phot(imseg,skyfile=daoout,coords=daoout,output=photout,interactive='no')
    psf(imseg,photfile=daoout,pstfile=photout,psfimage=psfout,opstfile=opstout,groupfile=groupout,interactive='no')
    starlist(starfieldsect,nstars=1000/nsegs,xmin=Xmin,xmax=Xmax,ymin=Ymin,ymax=Ymax,spatial=spatialDist,minmag=minMag,maxmag=maxMag)
    subprocess.call(['cp',starfieldsect,'/home/mj1e16/starlists/starfield_{}_{}.dat'.format(section,minMag)])
    try:
        mkobjects(diffimage,output=diffPlusSim,objects=starfieldsect,gain=110,rdnoise=127,star=psfout+'.fits')
        
    except:
        try:
            #subprocess.call(['rm','/home/mj1e16/Simages/testsimage*'])
            mkobjects(diffimage,output=diffPlusSim,objects=starfieldsect,gain=110,rdnoise=127,star=goodpsf)
            print('Waring: Section {} with bad PSF, {} used inplace'.format(section,goodpsf))
        except:
            print('ERROR: PSF for Section {} should be made manually'.format(section))
    
    return psfout+'.fits'

In [26]:
def findPSFandMkObjects(imageName,diffImage,SImage,extension,minMag,maxMag,imageshape=[1015., 1085.],pixelsize=3.98):
    
    print(imageName)
    
    os.chdir(irafDir)
    
    xlength = imageshape[1]
    ylength = imageshape[0]
    nsegs = 4
    
    xsegment = xlength/nsegs
    ysegment = ylength/nsegs
    PSF = []
    goodpsf=''
    for xsegs in range(nsegs):
        lowx = int(xsegs*xsegment)+15
        highx = int(lowx+xsegment)+15
        psf = []
        for ysegs in range(0,nsegs):
            lowy = int(ysegs*ysegment)+15
            highy = int(lowy+ysegment)+15
            
            
            # stars are being lost from images in xseg=0
            SImage= imageDir+'testsimage_{}_{}_{}.fits'.format(xsegs,ysegs,minMag)
            
            goodpsf = mkobjectspluspsf(imageName,diffImage,SImage,extension,'{}_{}'.format(xsegs,ysegs),minMag,maxMag,Xmax=highx,Xmin=lowx,Ymax=highy,Ymin=lowy,nsegs=nsegs**2,goodpsf=goodpsf)
            
            b = diffImage.find('{}_alt'.format(minMag))
            if b == -1:
                diffImage = diffImage[:-8]+'{}_alt.fits'.format(minMag)
            #try:
            hdu = fits.open(SImage) #,ignore_missing_end=True)
            imdata = hdu[0].data
            hdu = fits.PrimaryHDU(imdata)
            hdu.writeto(diffImage,clobber=True)
            print('find me')
            print(SImage)
            print(diffImage)
#             except:
#                 print('bad at section {} {}'.format(xsegs,ysegs))

            

            
    return 'complete!'

In [9]:
def cleanDirectories(dirlist):
    # clean iraf output and input images
    # make sure to have no needed files in these directories
    for directory in dirlist:
        dirfiles = os.listdir(directory)
        for eachFile in dirfiles:
            #print('rm',directory+eachFile)
            subprocess.call(['rm',directory+eachFile])

In [10]:
def stitchStarList(nsegs,mag,starDir='/home/mj1e16/starlists/'):
    with open(starDir+'starfield_0_0_{}.dat'.format(mag)) as f:
        lines = f.readlines()
    bigstring = ['# X_POS Y_POS MAG\n']
    bigstring.extend(lines[0:18])
    for x in range(nsegs):
        for y in range(nsegs):
            with open(starDir+'starfield_{}_{}_{}.dat'.format(x,y,mag)) as f:
                lines = f.readlines()
            bigstring.extend(lines[18:])


    with open('/home/mj1e16/keplerPhotometry/starlistFull_{}.dat'.format(mag),'w') as f:
        f.write(''.join(bigstring))


In [30]:
cleanDirectories([outDir,imageDir]) # cleans iraf output files, will remove all files from these directories

In [28]:
ext =  44
medianNum = 2
diffImage = '/home/mj1e16/Simages/diff_{}_{}.fits'.format(ext,medianNum)
diffImage = '/home/mj1e16/Simages/regIm_79.fits[1]'

In [33]:
run = 'not complete'
ext =  79
medianNum = 2
#diffImage = '/home/mj1e16/Simages/diff_{}_{}.fits'.format(ext,medianNum)
diffImage = '/home/mj1e16/Simages/regIm_79.fits[1]'
for mag in range(-7,0):
    cleanDirectories([outDir,imageDir]) # cleans iraf output files, will remove all files from these directories
    for x in range(5):
        try:
            run = findPSFandMkObjects(imagename,diffImage,SImage,ext,mag,(mag+1))
            if run == 'complete!':
                break
        except:
            pass
    dirlist = os.listdir('/home/mj1e16/starlists/')
    dirlist = [x for x in dirlist if '.dat' in x]
    for x in dirlist:
        subprocess.call(['mv',starlistDir+x,starlistDir+'{}/{}/'.format(ext,medianNum)])

/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the sky annulus in scale units (10.) (CR or value): 	New width of the sky annulus: 10. scale units 10. pi

Killing IRAF task `psf'


/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the sky annulus in scale units (10.) (CR or value): 	New width of the sky annulus: 10. scale units 10. pi

Killing IRAF task `mkobjects'
Killing IRAF task `mkobjects'


ERROR: PSF for Section 0_0 should be made manually
find me
/home/mj1e16/Simages/irafImages/testsimage_0_0_-7.fits
/home/mj1e16/Simages/regIm_79-7_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the s

...

ERROR: PSF for Section 3_0 should be made manually
find me
/home/mj1e16/Simages/irafImages/testsimage_3_0_-1.fits
/home/mj1e16/Simages/regIm_79-1_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the s

Killing IRAF task `mkobjects'
Killing IRAF task `mkobjects'


ERROR: PSF for Section 3_1 should be made manually
find me
/home/mj1e16/Simages/irafImages/testsimage_3_1_-1.fits
/home/mj1e16/Simages/regIm_79-1_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the s

Killing IRAF task `mkobjects'
Killing IRAF task `mkobjects'


ERROR: PSF for Section 3_2 should be made manually
find me
/home/mj1e16/Simages/irafImages/testsimage_3_2_-1.fits
/home/mj1e16/Simages/regIm_79-1_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the s

In [29]:
mag = -7
for x in range(5):
        try:
            run = findPSFandMkObjects(imagename,diffImage,SImage,ext,mag,(mag+1))
            if run == 'complete!':
                break
        except:
            pass

/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the sky annulus in scale units (10.) (CR or value): 	New width of the sky annulus: 10. scale units 10. pi



Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the sky annulus in scale units (10.) (CR or value): 	New width of the sky annulus: 10. scale units 10. pixels
File/list of aperture radii in scale units (3.) (CR or value): 	Aperture radius 1: 3. scale units 3. pixels
Standard deviation of background in counts (0.) (CR or value): 	New standard deviation of background: 0. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Analytic psf function(s) (gauss): 	Analytic psf function(s): gauss
Order of variable psf (0): 	Order of variable psf: 0
Psf radius in scale units (11.): 	New psf radius: 11. scale units 11. pixels
Fitting r

Star 3 has been added to the PSF star list
	X:   89.64 Y:   64.86  Mag:  -0.345  Dmin: 166.6266  Dmax: 85378.46
Star 4 has been added to the PSF star list
	X:   71.82 Y:   69.68  Mag:  -1.303  Dmin: 169.0527  Dmax: 189158.5
Star 5 has been added to the PSF star list
	X:   31.40 Y:   80.72  Mag:  -1.084  Dmin: 164.2826  Dmax: 163347.4
Star 6 has been added to the PSF star list
	X:   85.93 Y:   97.76  Mag:  -1.070  Dmin: 168.7512  Dmax: 167009.5
Star 7 has been added to the PSF star list
	X:   95.61 Y:  104.77  Mag:  -1.195  Dmin: 165.0025  Dmax: 168279.8
Star 8 has been added to the PSF star list
	X:  221.46 Y:  138.81  Mag:  -1.438  Dmin: 165.4123  Dmax: 176034.6
Star 9 has been added to the PSF star list
	X:   62.59 Y:  184.94  Mag:  -0.026  Dmin: 161.3456  Dmax: 62784.11
Star 10 has been added to the PSF star list
	X:   28.00 Y:  200.82  Mag:  -1.232  Dmin: 161.188  Dmax: 165588.5
Star 11 has been added to the PSF star list
	X:   33.50 Y:  254.66  Mag:  -1.142  Dmin: 161.7228  Dmax: 


FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the sky annulus in scale units (10.) (CR or value): 	New width of the sky annulus: 10. scale units 10. pixels
File/list of aperture radii in scale units (3.) (CR or value): 	Aperture radius 1: 3. scale units 3. pixe

Star 11 has been added to the PSF star list
	X:   92.08 Y:  228.70  Mag:  -0.881  Dmin: 174.2743  Dmax: 174483.1

11 PSF stars read from /home/mj1e16/iraf/outputs/44_2_3photone


Fitting function gauss    norm scatter: 0.3020946

Analytic PSF fit 
    Function: gauss  X: 133.5  Y: 142.5  Height: 70310.66  Psfmag: 11.792
    Par1: 1.026136  Par2: 2.202869

Computed 1 lookup table(s)

Writing PSF image /home/mj1e16/iraf/outputs/44_2_3psfone.fits
Writing output PSF star list /home/mj1e16/iraf/outputs/44_2_3optsone
Writing output PSF star group file /home/mj1e16/iraf/outputs/44_2_3groupfileone
find me
/home/mj1e16/Simages/irafImages/testsimage_2_3_-7.fits
/home/mj1e16/Simages/regIm_79-7_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigm

In [14]:
for mag in range(-7,0):
    stitchStarList(4,mag,starDir='/home/mj1e16/starlists/79/2/')

In [17]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'global color=lightgreen\nimage\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},5)\n'.format(xcoords[x],ycoords[x])
        fileName = fileNameBase + str(tables)+'.reg'
        with open(fileName,'w') as f:
            f.write(bigString)
        print(fileName)


In [15]:
data = ascii.read('/home/mj1e16/keplerPhotometry/starlistFull.dat')

In [16]:
makeDS9RegFile([data],'/home/mj1e16/Simages/teststarlist','iraf')

/home/mj1e16/Simages/teststarlist0.reg
['44_2_0photone', 'logfile_63_x2_y7', '44photone', 'logfile_9_x8_y13', 'logfile_63_x13_y6', 'starfield_0_0.dat', 'logfile_44_x0_y10', 'logfile_9_x9_y3', '44_3_3photone', 'logfile_44_x13_y2', 'mkObjects.ipynb', 'logfile_44_x13_y12', 'logfile_63_x1_y10', 'logfile_63_x0_y10', 'test19.fits', 'logfile_44_x8_y4', 'login.cl', 'logfile_44_x6_y7', '44_0_4groupfileone', 'logfile_9_x13_y11', '44_0_0photone', 'logfile_9_x9_y2', 'removeStarfield.sh', 'logfile_63_x7_y9', 'logfile_9_x5_y7', 'logfile_63_x13_y13', 'logfile_63_x9_y12', 'test4.fits', 'logfile_63_x6_y5', 'logfile_9_x12_y1', 'imaginary1000.fits', 'logfile_63_x4_y3', 'logfile_9_x12_y2', 'test6.fits', '44_4_1daofindone', 'logfile_44_x6_y9', 'logfile_9_x12_y9', 'logfile_9_x14_y7', 'starfield_1_3.dat', 'logfile_9_x11_y9', 'aaa.sh', 'logfile_44_x8_y9', 'logfile_9_x7_y11', 'logfile_44_x14_y8', 'logfile_63_x7_y2', 'logfile_63_x2_y0', 'logfile_63_x6_y12', 'logfile_9_x0_y9', 'logfile_9_x8_y12', 'logfile_44_x6_

In [2]:
for x in range(-7,0):
    print(x)

-7
-6
-5
-4
-3
-2
-1
